In [ ]:
import xarray as xr
import pystac_client
import odc.stac
from odc.geo.geobox import GeoBox
from dask.diagnostics import ProgressBar
from rasterio.enums import Resampling
import geopandas as gpd
import planetary_computer as pc

In [ ]:
sa_bounds = gpd.read_file("./data/sa_bounds.gpkg")

In [ ]:
sa_bounds.plot()

In [ ]:
minx,miny,maxx,maxy = sa_bounds.total_bounds

In [ ]:
geom = {
    'type': 'Polygon',
    'coordinates': [[
       [minx, miny],
       [minx, maxy],
       [maxx, maxy],
       [maxx, miny],
       [minx, miny]
    ]]
}


In [ ]:
items = pystac_client.Client.open(
    'https://planetarycomputer.microsoft.com/api/stac/v1'
).search(
    bbox=[minx,miny,maxx,maxy],
    collections=["sentinel-2-l2a"],
    datetime="2020-03-01/2020-03-31"
).item_collection()

print(len(items), "scenes found")

In [ ]:
bbox_hart = sa_bounds.to_crs("EPSG:9221") 

In [ ]:
dx = 1000
geobox = GeoBox.from_bbox(bbox_hart.total_bounds, crs="EPSG:9221", resolution=dx)

In [ ]:
geobox

In [ ]:
# lazily combine items
ds_odc = odc.stac.load(
    items,
    bands=["SCL", "B04", "B03", "B02"],
    chunks={'time': 1, 'x': 100, 'y': 100},
    geobox=geobox,
    resampling="bilinear",
    patch_url=pc.sign
)

In [ ]:
ds_odc

In [ ]:
with ProgressBar():
    ds_odc.load()

In [ ]:
ds_odc

In [ ]:
def is_valid_pixel(data):
    return ((data > 3) & (data < 7)) | (data==11)

ds_odc['valid'] = is_valid_pixel(ds_odc.SCL)
ds_odc.valid.sum("time").plot()

In [ ]:
ds_median = ds_odc.where(ds_odc.valid).median(dim="time")

In [ ]:
rgb_median = (
    ds_median[['B04', 'B03', 'B02']]
    .to_dataarray(dim="band")
    .transpose(..., "band")
)
(rgb_median / rgb_median.max() * 2).plot.imshow(rgb="band", figsize=(10, 8))


In [ ]:
items = pystac_client.Client.open(
    'https://planetarycomputer.microsoft.com/api/stac/v1'
).search(
    bbox=[minx,miny,maxx,maxy],
    collections=["nasadem"],
).item_collection()
dem_odc = odc.stac.load(
    items,
    bands=["elevation"],
    chunks={'time': 1, 'x': 100, 'y': 100},
    geobox=geobox,
    resampling="bilinear",
    patch_url=pc.sign
)

In [ ]:
with ProgressBar():
    dem_odc.load()

In [ ]:
ds_median["elev"] = dem_odc.isel(time = 0).elevation

In [ ]:
ds_median